# Direct beam iteration

In [ ]:
import scipp as sc
import numpy as np
import sans
import contrib

In [ ]:
def setup_masks(data):
    masks = {}
    bad_straws = sc.Variable(dims=['spectrum'], shape=sample.coords['spectrum'].shape, dtype=sc.dtype.bool)
    mask = sc.Variable(value=True)
    bad_straws[loki.straw(tube=0, straw=6)] |= mask
    bad_straws[loki.straw(tube=1, straw=2)] |= mask
    bad_straws[loki.straw(tube=5, straw=1)] |= mask
    for straw in 1,2,3,4,5:
        bad_straws[loki.straw(tube=12, straw=straw)] |= mask
    
    tof = data.coords['tof']
    data.masks['bins'] = sc.less(tof['tof',1:], 1500.0 * sc.units.us) | \
                         (sc.greater(tof['tof',:-1], 17500.0 * sc.units.us) & \
                          sc.less(tof['tof',1:], 19000.0 * sc.units.us))
    pos = sc.neutron.position(data)
    x = sc.geometry.x(pos)
    y = sc.geometry.y(pos)
    masks['bad-straws'] = bad_straws
    masks['beam-stop'] = sc.less(sc.sqrt(x*x+y*y), 0.045 * sc.units.m)
    masks['tube-ends'] = sc.greater(sc.abs(x), 0.36 * sc.units.m) # roughly all det IDs listed in original
    #MaskDetectorsInShape(Workspace=maskWs, ShapeXML=self.maskingPlaneXML) # irrelevant tiny wedge?
    return masks

In [ ]:
def fit_gauss_coil(data, flat_background):
    import functools
    from scipp.compat.mantid import fit
    import gauss_coil_fit1
    model = "polyGaussCoil"
    params = f"I0=60.0,Rg=50.0,Mw_by_Mn=1.02,Background={flat_background.value}"
    ties = "Rg=50.0,Mw_by_Mn=1.02"
    def fit_layer(i):
        return fit(data['layer', i],
                   mantid_args={'Function':f'name={model},{params}', 'Ties':ties})
    params, diff = zip(*map(fit_layer, range(data.coords['layer'].shape[0])))
    concat = functools.partial(sc.concatenate, dim='layer')
    return functools.reduce(concat, params), functools.reduce(concat, diff)

In [ ]:
def normalize_and_subtract(sample, background):
    sample_norm = sample['data'] / sample['norm']
    background_norm = background['data'] / background['norm']
    return sample_norm - background_norm

def q_range_mask(wavelength):
    # Ratio of reduced_band / reduced is averaged (integrated) over wavelength-dependent interval
    # Use a mask, so we can just use `sc.mean`.
    # Note that in the original script Integration(...)/delta_Q is used, but this
    # is biased due to log binning, maybe on purpose...?
    inv_w = sc.reciprocal(wavelength)
    dim = inv_w.dims[0]
    d_inv_w = sc.concatenate(inv_w[dim,1:].copy(), inv_w[dim,:-1].copy(), 'bound') - inv_w[dim,-1]
    q_range = qlongW + d_inv_w * (qshortW - qlongW) / (inv_w[dim,0] - inv_w[dim,-1])
    qmin = q_range['bound',0]
    qmax = q_range['bound',1]
    return sc.greater(qmin, q_bins['Q',:-1]) | sc.less(qmax, q_bins['Q',1:])

def interpolate_cubic_spline(data, x, dim):
    import functools
    from scipy import interpolate
    def interpolate_layer(i):
        tck = interpolate.splrep(contrib.midpoints(data.coords[dim], dim).values, data['layer',i].values)
        # TODO uncertainties
        return sc.Variable(dims=[dim], values=interpolate.splev(x.values, tck))
    y = map(interpolate_layer, range(data.coords['layer'].shape[0]))
    concat = functools.partial(sc.concatenate, dim='layer')
    return sc.DataArray(data=functools.reduce(concat, y), coords={dim:x.copy()})

def to_q_by_wavelength(data, transmission, direct_beam, wavelength_bands):
    wav = sans.to_wavelength(data=data,
                             transmission=transmission,
                             direct_beam=direct_beam,
                             masks=setup_masks(data),
                             wavelength_bins=wavelength_bins)
    return sans.reduce_by_wavelength(wav, q_bins, groupby='layer', wavelength_bands=wavelength_bands)

def direct_beam_iteration(direct_beam, layers, wavelength_bands, flat_background=0.25*sc.units.one):
    print('start iteration')
    direct_beam_by_pixel = sc.choose(layers, choices=direct_beam, dim='layer')
    sample_q_lambda = to_q_by_wavelength(data=sample, transmission=sample_trans,
                                         direct_beam=direct_beam_by_pixel,
                                         wavelength_bands=wavelength_bands)
    background_q_lambda = to_q_by_wavelength(data=background, transmission=background_trans,
                                             direct_beam=direct_beam_by_pixel,
                                             wavelength_bands=wavelength_bands)

    # reduced by wavelength band
    reduced_by_wavelength = normalize_and_subtract(sample_q_lambda, background_q_lambda)
    
    # sum wavelength bands to reduce for full range
    sample_q1d = sc.sum(sample_q_lambda, 'wavelength')
    background_q1d = sc.sum(background_q_lambda, 'wavelength')
    reduced = normalize_and_subtract(sample_q1d, background_q1d)
    
    params, diff = fit_gauss_coil(contrib.select_bins(reduced, 'Q', qlongW['bound', 0], qshortW['bound', 1]),
                                  flat_background=flat_background)
    scale = I0_expected / params['parameter', 0].data
    reduced_by_wavelength *= scale
    reduced *= scale
    direct_beam = direct_beam / scale
    ratio = (reduced_by_wavelength - flat_background) / (reduced - flat_background)
    ratio.masks['Q-range-mask'] = q_range_mask(wavelength_bands)
    norm = 1.0*sc.units.one + damp*(sc.mean(ratio, 'Q') - 1.0*sc.units.one)
    # TODO original uses scale for interpolation, but is this necessary if we don't deal with histogram?
    # d_w = wavelength['wavelength',1:]-wavelength['wavelength',:-1]
    # scale = 1.0*sc.units.angstrom + damp*(norm*d_w-1.0*sc.units.angstrom)
    splined = interpolate_cubic_spline(norm, direct_beam.coords['wavelength'], 'wavelength')
    direct_beam *= splined
    
    # store some information for inspecting intermediate terms
    direct_beam.attrs['fit'] = sc.Variable(value=diff)
    direct_beam.attrs['spline'] = sc.Variable(value=splined)
    
    return direct_beam

In [ ]:
import dataconfig # run make_config.py to create this

path = dataconfig.data_root
direct_beam_file = 'DirectBeam_20feb_full_v3.dat' # same as DirectBeam_28Apr2020.dat
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'
sample_run_number = 49338
sample_transmission_run_number = 49339
background_run_number = 49334
background_transmission_run_number = 49335

def load_larmor(run_number):
    return sc.neutron.load(filename=f'{path}/LARMOR000{run_number}.nxs')

def load_rkh(filename):
    return sc.neutron.load(
           filename=filename,
           mantid_alg='LoadRKH',
           mantid_args={'FirstColumnValue':'Wavelength'})

In [ ]:
%%time
sample_trans = load_larmor(sample_transmission_run_number)
sample = load_larmor(sample_run_number)
background_trans = load_larmor(background_transmission_run_number)
background = load_larmor(background_run_number)

## Visualizations

### Bad straws

In [ ]:
from scipp.plot import plot
from loki import LoKI
loki = LoKI()
pixel_counts = sc.sum(sample, 'tof') # sum is optional, could also keep TOF
pixel_counts = loki.to_logical_dims(pixel_counts)
straw_counts = sc.sum(pixel_counts, 'pixel')
plot(straw_counts, log=True, vmin=1, vmax=10)

### X-Y projection

In [ ]:
from scipp.plot import plot
plot(sans.project_xy(sc.sum(sample,'tof')))
plot(sans.project_xy(sample))

## Reduction and direct-beam iteration

In [ ]:
%%time
dtype = sample.coords['position'].dtype
sample_pos_offset = sc.Variable(value=[0.0, 0.0, 0.30530], unit=sc.units.m, dtype=dtype)
bench_pos_offset = sc.Variable(value=[0.0, 0.001, 0.0], unit=sc.units.m, dtype=dtype)
for item in [sample, sample_trans, background, background_trans]:
    item.coords['sample-position'] += sample_pos_offset
    item.coords['position'] += bench_pos_offset

In [ ]:
q_bins = sc.Variable(
    dims=['Q'],
    unit=sc.units.one/sc.units.angstrom,
    values=np.geomspace(0.007999, 0.6, num=55))
wavelength_bins = sc.Variable(
    dims=['wavelength'],
    unit=sc.units.angstrom,
    values=np.geomspace(1.0, 12.9, num=110))
wavelength_bands = sc.Variable(
    dims=['wavelength'],
    unit=sc.units.angstrom,
    values=[1.0,1.4,1.8,2.2,3.0,4.0,5.0,7.0,9.0,11.0,12.9])

In [ ]:
I0_expected = 55.77 * sc.units.one

In [ ]:
qlongW = sc.Variable(dims=['bound'], unit=q_bins.unit, values=[0.008, 0.05])
qshortW = sc.Variable(dims=['bound'], unit=q_bins.unit, values=[0.1, 0.22])
damp = 1.0*sc.units.one

In [ ]:
%%time
from loki import LoKI
loki = LoKI()
layers = loki.layers()
n_layer = sc.max(layers).value + 1
direct_beam = load_rkh(filename=f'{path}/{direct_beam_file}')
# Use same starting value for all layers
direct_beam = sc.Variable(dims=['layer'], values=np.ones(n_layer)) * direct_beam
direct_beam.coords['layer'] = sc.Variable(dims=['layer'], dtype=sc.dtype.int32, values=np.arange(n_layer))

In [ ]:
direct_beams = [direct_beam]
for i in range(4):
    direct_beams.append(direct_beam_iteration(direct_beams[-1],
                                              layers=layers,
                                              wavelength_bands=wavelength_bands))
direct_beams = sc.Dataset({f'iteration-{i}':item for i, item in enumerate(direct_beams)})

In [ ]:
from scipp.plot import plot
plot(direct_beams['layer',3])

In [ ]:
layer_index = 0
for i in 1,2,4:
    plot(direct_beams[f'iteration-{i}'].attrs['fit'].value['layer', layer_index])

In [ ]:
plot(direct_beams['iteration-4'].attrs['spline'].value['wavelength',:130], collapse='wavelength')